In [3]:
from src.file_converter import (
    convert_doc_with_image_annotation,
    chunk_markdown_by_headers,
    export_to_md_with_image_ref,
)

In [4]:
markdownDocument = convert_doc_with_image_annotation("docs/test-document-1.pdf")
# markdownString = markdownDocument.document.export_to_markdown()

2025-12-14 21:31:35,098 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2025-12-14 21:31:38,644 - INFO - Going to convert document batch...
2025-12-14 21:31:38,645 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 5bc1909d1e8430ba75d1da6a9ed3bb9e
2025-12-14 21:31:38,696 - INFO - Loading plugin 'docling_defaults'
2025-12-14 21:31:38,715 - INFO - Registered picture descriptions: ['vlm', 'api']
2025-12-14 21:31:38,735 - INFO - Loading plugin 'docling_defaults'
2025-12-14 21:31:38,752 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2025-12-14 21:31:38,940 - INFO - rapidocr cannot be used because onnxruntime is not installed.
2025-12-14 21:31:38,941 - INFO - easyocr cannot be used because it is not installed.
2025-12-14 21:31:40,379 - INFO - Accelerator device: 'cpu'
[INFO] 2025-12-14 21:31:40,442 [RapidOCR] base.py:22: Using engine_name: torch
[INFO] 2025-12-14 21:31:40,471 [RapidOCR] device_config.py:50: Usi

In [5]:
chunkedMarkdownDocument = chunk_markdown_by_headers(markdownDocument.document.export_to_markdown())
textsList = [ chunk.page_content for chunk in chunkedMarkdownDocument]
for text in textsList:
	print(text)

<!-- image -->
## Docling Technical Report
## Version 1.0  
Christoph Auer Maksym Lysak Ahmed Nassar Michele Dolfi Nikolaos Livathinos Panos Vagenas Cesar Berrospi Ramis Matteo Omenetti Fabian Lindlbauer Kasper Dinkla Lokesh Mishra Yusik Kim Shubham Gupta Rafael Teixeira de Lima Valery Weber Lucas Morin Ingmar Meijer Viktor Kuropiatnyk Peter W. J. Staar  
AI4K Group, IBM Research R¨ uschlikon, Switzerland
## Abstract  
This technical report introduces Docling , an easy to use, self-contained, MITlicensed open-source package for PDF document conversion. It is powered by state-of-the-art specialized AI models for layout analysis (DocLayNet) and table structure recognition (TableFormer), and runs efficiently on commodity hardware in a small resource budget. The code interface allows for easy extensibility and addition of new features and models.
## 1 Introduction  
Converting PDF documents back into a machine-processable format has been a major challenge for decades due to their huge vari

In [6]:

# markdownWithImageRef = export_to_md_with_image_ref(markdownDocument)
picturesList = markdownDocument.document.pictures
imageAnnotationsList = [el.annotations[0].text for el in picturesList if len(el.annotations)>0]
print(imageAnnotationsList)

['The image presents a detailed, step-by-step process for extracting and processing data from a PDF document. A PDF document, denoted by the Adobe logo, serves as the starting point. The document then undergoes parsing into individual PDF pages. These pages are then fed into a model pipeline, represented by a dotted line, which comprises four key stages: \n1. **OCR (Optical Character Recognition)**, \n2. **Layout Analysis**, \n3. **Table Structure**. \nFollowing the model pipeline, the document pages are reassembled and subjected to post-processing. The final output stage involves serializing the processed data into either JSON or Markdown format. This workflow outlines a structured approach to converting PDF documents into more usable and machine-readable formats.']


C:\Users\ASUS\AppData\Local\Temp\ipykernel_16916\1864593035.py:3: DeprecationWarning: Field `annotations` is deprecated; use `meta` instead.
  imageAnnotationsList = [el.annotations[0].text for el in picturesList if len(el.annotations)>0]


In [7]:
# Separating tables
hasSeparatedTablesFromDocument = markdownDocument.document.tables
tablesList = [table.export_to_html(doc=markdownDocument.document) for table in hasSeparatedTablesFromDocument]

In [8]:
# Text and Table Summaries
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [9]:
promptText = """
You are an assistant tasked with summarizing tables and text.
Give a concise summary of the table or text.

Respond only with the summary, no additional comment.
Do not start your message by saying "Here is a summary" or anything like that.
Just give the summary as it is.

Table or text chunk: {element}
"""
prefixPrompt = ChatPromptTemplate.from_template(promptText)
model = ChatGroq(temperature=0.5,model="llama-3.1-8b-instant")
summarizerChain = {"element": lambda x:x} | prefixPrompt | model | StrOutputParser()

In [10]:
textSummaries = summarizerChain.batch(textsList, {'max_concurrency': 3})

tableSummaries = summarizerChain.batch(tablesList, {'max_concurrency': 3})

2025-12-14 21:32:19,910 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:19,917 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:19,944 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:20,220 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:20,254 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:20,284 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:20,433 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:20,485 - INFO - HTTP Request: POST https://api.groq.com/openai/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-14 21:32:20,613 - INFO - HTTP Request: POST http

In [ ]:
print(textSummaries)
print(tableSummaries)

In [11]:
# Setting up the vector store
import getpass
import os
from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from langchain_google_genai import GoogleGenerativeAIEmbeddings

if not os.environ.get("GOOGLE_API_KEY"):
	os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

embeddings = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

QDRANT_URL = os.getenv("QDRANT_URL")
QDRANT_API_KEY = os.getenv("QDRANT_API_KEY")
client = QdrantClient(url=QDRANT_URL, api_key=QDRANT_API_KEY, timeout=30)

vector_size = len(embeddings.embed_query("sample text"))

if not client.collection_exists("conversational_tutoring"):
    client.create_collection(
        collection_name="conversational_tutoring",
        vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
    )
vectorStore = QdrantVectorStore(
    client=client,
    collection_name="conversational_tutoring",
    embedding=embeddings,
)

2025-12-14 21:32:31,701 - INFO - HTTP Request: GET https://5e0668f4-54a2-4602-9c11-634e756c29c0.europe-west3-0.gcp.cloud.qdrant.io:6333 "HTTP/1.1 200 OK"
2025-12-14 21:32:32,597 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-12-14 21:32:33,267 - INFO - HTTP Request: GET https://5e0668f4-54a2-4602-9c11-634e756c29c0.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/conversational_tutoring/exists "HTTP/1.1 200 OK"
2025-12-14 21:32:33,502 - INFO - HTTP Request: GET https://5e0668f4-54a2-4602-9c11-634e756c29c0.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/conversational_tutoring "HTTP/1.1 200 OK"
2025-12-14 21:32:33,964 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"


In [42]:
from typing import Any, List
import uuid
from langchain_core.stores import InMemoryStore
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
# The storage layer for the parent documents
store = InMemoryStore()
idKey = "doc_id"

class MultiVectorRetriever(BaseRetriever): # langchain compatible implementation
    vectorstore: Any
    docstore: Any
    id_key: str
    k: int = 10

    def _get_relevant_documents(self, query: str) -> List[Document]:
        sub_docs = self.vectorstore.similarity_search(query, k=self.k)

        parent_ids = []
        for doc in sub_docs:
            if self.id_key not in doc.metadata:
                raise ValueError(f"Missing `{self.id_key}` in document metadata")
            parent_ids.append(doc.metadata[self.id_key])

        # Deduplicate (preserve order)
        seen = set()
        parent_ids = [x for x in parent_ids if not (x in seen or seen.add(x))]

        parent_docs = self.docstore.mget(parent_ids)
        return [doc for doc in parent_docs if doc is not None]

# The retriever (empty to start)
retriever = MultiVectorRetriever(
    vectorstore=vectorStore,
    docstore=store,
    id_key=idKey,
)

In [43]:
# Add texts
docIds = [str(uuid.uuid4()) for _ in textsList]
summaryTexts = [
    Document(page_content=summary, metadata={idKey: docIds[i]}) for i, summary in enumerate(textSummaries)
]
retriever.vectorstore.add_documents(summaryTexts)
retriever.docstore.mset(list(zip(docIds, textsList)))

# Add tables
tableIds = [str(uuid.uuid4()) for _ in tablesList]
summaryTables = [
    Document(page_content=summary, metadata={idKey: tableIds[i]}) for i, summary in enumerate(tableSummaries)
]
retriever.vectorstore.add_documents(summaryTables)
retriever.docstore.mset(list(zip(tableIds, tablesList)))

# Add image summaries
imgIds = [str(uuid.uuid4()) for _ in imageAnnotationsList]
summaryImgs = [
    Document(page_content=summary, metadata={idKey: imgIds[i]}) for i, summary in enumerate(imageAnnotationsList)
]
retriever.vectorstore.add_documents(summaryImgs)
retriever.docstore.mset(list(zip(imgIds, imageAnnotationsList)))

2025-12-14 22:52:27,603 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-12-14 22:52:30,172 - INFO - HTTP Request: PUT https://5e0668f4-54a2-4602-9c11-634e756c29c0.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/conversational_tutoring/points?wait=true "HTTP/1.1 200 OK"
2025-12-14 22:52:30,617 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-12-14 22:52:30,890 - INFO - HTTP Request: PUT https://5e0668f4-54a2-4602-9c11-634e756c29c0.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/conversational_tutoring/points?wait=true "HTTP/1.1 200 OK"
2025-12-14 22:52:31,330 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-12-14 22:52:31,593 - INFO - HTTP Request: PUT https://5e0668f4-54a2-4602-9c11-634e7

In [50]:
# Testing out the retrieval
docs = retriever._get_relevant_documents(
    "comparison")

2025-12-14 22:54:47,357 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-embedding-001:batchEmbedContents "HTTP/1.1 200 OK"
2025-12-14 22:54:48,542 - INFO - HTTP Request: POST https://5e0668f4-54a2-4602-9c11-634e756c29c0.europe-west3-0.gcp.cloud.qdrant.io:6333/collections/conversational_tutoring/points/query "HTTP/1.1 200 OK"


In [51]:
for doc in docs:
    print(str(doc) + "\n\n" + "-" * 80)

<table><caption><div class="caption">Table 1: Runtime characteristics of Docling with the standard model pipeline and settings, on our test dataset of 225 pages, on two different systems. OCR is disabled. We show the time-to-solution (TTS), computed throughput in pages per second, and the peak memory used (resident set size) for both the Docling-native PDF backend and for the pypdfium backend, using 4 and 16 threads.</div></caption><tbody><tr><td>CPU</td><th>Thread budget</th><th colspan="3">native backend</th><th colspan="3">pypdfium backend</th></tr><tr><td></td><td></td><th>TTS</th><th>Pages/s</th><th>Mem</th><th>TTS</th><th>Pages/s</th><th>Mem</th></tr><tr><th>Apple M3 Max (16 cores)</th><td>4 16</td><td>177 s 167 s</td><td>1.27 1.34</td><td>6.20 GB</td><td>103 s 92 s</td><td>2.18 2.45</td><td>2.56 GB</td></tr><tr><th>Intel(R) Xeon E5-2690</th><td>4 16</td><td>375 s 244 s</td><td>0.60 0.92</td><td>6.16 GB</td><td>239 s 143 s</td><td>0.94 1.57</td><td>2.42 GB</td></tr></tbody></tabl